# bert model

In [49]:
import os
import ast
import torch
import time
import inspect
import pandas as pd
import numpy as np
import torch.nn as nn
from torchinfo import summary
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import multiprocessing as mp
from transformers import AutoTokenizer

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# tokenization of dataset
multiprocessing in seperate file

In [ ]:
import os
import numpy as np
import pandas as pd
import multiprocessing as mp
from transformers import AutoTokenizer


#read data
def load():
    df = pd.read_csv('C:\\Users\\vd00r\\OneDrive\\Documenten\\GitHub\\MadManAndMachines\\pythonProject\\text_data.csv', sep=',')
    #turn string to list
    for i, id in enumerate(df['codes']):
        df['codes'][i] = eval(id)
    #drop data
    df = df.drop(['Unnamed: 0'],axis = 1)
    return(df.to_numpy())

#multiprocessing
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
cpu_c = os.cpu_count()//2 

def transform(fw):
    print(fw[0])
    tok = tokenizer.encode_plus(fw[0])
    return tok,fw[1]

def proc(fw):
    with mp.Pool(cpu_c) as pool:
        tokenized_text = []
        labels = []
        for data in pool.imap(transform,fw,chunksize=16):
            tokenized_text.append(data[0])
            labels.append(data[1])
        return tokenized_text,labels

if __name__ == '__main__':
    df = load()
    #print(df)
    a = proc(df)
    #print(a[0]['input_ids'])
    d = {"inputs": a[0],"labels": a[1]} 
    array = pd.DataFrame(data = d).to_csv('dataset.csv',index=False)
    print(array)

C:\Users\vd00r\AppData\Local\Temp\ipykernel_44024\3749047469.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['codes'][i] = eval(id)
C:\Users\vd00r\AppData\Local\Temp\ipykernel_44024\3749047469.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['codes'][i] = eval(id)
C:\Users\vd00r\AppData\Local\Temp\ipykernel_44024\3749047469.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['codes'][i] = eval(id)
C:\Users\vd00r\AppD

# Dataset class

In [3]:

df = pd.read_csv('dataset.csv', sep=',', converters={'labels': pd.eval})
print(df['labels'][0])


[10, 27]


In [114]:
class political_ads_Dataset(Dataset):
    def __init__(self):
        self.data = pd.read_csv('dataset.csv', sep=',', converters={'labels': pd.eval})
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, idx):
        input = self.data['inputs'].iloc[idx]
        input = ast.literal_eval(input)['input_ids']
        input = torch.tensor(input)
        input = F.pad(input,(0,(512-input.shape[0])),"constant",0)
        label = self.data['labels'].iloc[idx]
        return input,label[0]

# model

In [42]:
class config:
    n_embd = 768
    n_classes = 32

In [119]:
class bert_classifier(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.config = config
        self.model_bert = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-uncased')
        self.lin = nn.Linear(config.n_embd,config.n_classes)
        #self.softmax = nn.softmax()
    def forward(self,x,target=None):
        x = self.model_bert(x)
        x = self.lin(x[1])
        if target is not None:
            loss = F.cross_entropy(x,target)
        return x,loss

In [124]:
model = bert_classifier(config).to(device)

Using cache found in C:\Users\vd00r/.cache\torch\hub\huggingface_pytorch-transformers_main


In [37]:
summary(model,input_data=torch.ones([1,512],dtype=torch.int32).to(device))

Layer (type:depth-idx)                                       Output Shape              Param #
bert_classifier                                              [1, 32]                   --
├─BertModel: 1-1                                             [1, 768]                  --
│    └─BertEmbeddings: 2-1                                   [1, 512, 768]             --
│    │    └─Embedding: 3-1                                   [1, 512, 768]             23,440,896
│    │    └─Embedding: 3-2                                   [1, 512, 768]             1,536
│    │    └─Embedding: 3-3                                   [1, 512, 768]             393,216
│    │    └─LayerNorm: 3-4                                   [1, 512, 768]             1,536
│    │    └─Dropout: 3-5                                     [1, 512, 768]             --
│    └─BertEncoder: 2-2                                      [1, 512, 768]             --
│    │    └─ModuleList: 3-6                                  --             

# training loop

In [8]:
#-----TRAINING-HYPERPARAMETERS-----#
max_lr = 3e-4
min_lr = max_lr * 0.01
warmup_steps = 100
max_steps = 3000
theta = 0.9
#-----TRAINING-HYPERPARAMETERS-----#

In [115]:
#-----BATCH-SIZE/GRADIENT-ACCUMILATION-----#
dataset = political_ads_Dataset()
train_set, test_set = torch.utils.data.random_split(dataset, [int(0.8*len(dataset)), len(dataset)-int(0.8*len(dataset))])
batch_size = 1024
B = 10
grad_accum_steps = batch_size // B
train_loader = DataLoader(train_set, batch_size=B, shuffle=True)
test_loader = DataLoader(test_set, batch_size=B, shuffle=True)
print(grad_accum_steps)
#-----BATCH-SIZE/GRADIENT-ACCUMILATION-----#

102


In [116]:
for i in range(2):
    train_features, train_labels = next(iter(train_loader))
    print(train_features.shape)
    print(train_labels)


torch.Size([10, 512])
tensor([ 0,  0, 29,  0,  0,  0,  0,  0,  0,  0])
torch.Size([10, 512])
tensor([10,  0, 10,  0,  0,  0,  0, 10,  0,  0])


In [120]:
#-----LEARNING-RATE-SCHEDULER-----#
def get_lr(it):
    if it < warmup_steps:
        return max_lr * (it+1) / warmup_steps
    if it > max_steps:
        return min_lr
    decay = it-warmup_steps
    return max_lr - decay * (max_lr/(max_steps-warmup_steps))
#-----LEARNING-RATE-SCHEDULER-----#

In [121]:
#-----PARAMETER-SETUP-----#
for param in model.parameters():
     param.requires_grad = False
for param in model.model_bert.pooler.parameters():
     param.requires_grad = True
for param in model.lin.parameters():
     param.requires_grad = True
#-----PARAMETER-SETUP-----#

In [122]:
layer_names = []
for param in model.named_parameters():
     layer_names.append(param[0])
layer_names.reverse()
#print(layer_names)

In [123]:
#-----GRADUAL-FREEZING----#
class gradual_freezing():
    def __init__(self):
        self.unlock_ratio = 0.083
        self.i = 12
    def check_unfreeze(self,it):
        ratio = it/max_steps
        #print(ratio)
        #print(self.unlock_ratio)
        if ratio > self.unlock_ratio and self.i != 0:
            self.unlock_ratio = self.unlock_ratio +  0.083
            self.i = self.i - 1
            print(self.i)
            for param in model.model_bert.encoder.layer[self.i].parameters():
                param.requires_grad = True
                #print(param[0])
            return True
        return False
#-----GRADUAL-FREEZING-----#
#a = gradual_freezing()
#for g in range(2000):
    #a.check_unfreeze(g)

In [125]:
#-----DISCRIMINATIVE-FINE-TUNING-----#
def disc_fine_tuning(lr):
    parameters = []
    list1 = []
    previous = layer_names[2].split('.')[2]
    lr_disc = lr
    for idx,name in enumerate(layer_names):
        if name.split(".")[0] == "lin":
            for p in model.named_parameters():
                if p[0] == name and p[1].requires_grad:
                    parameters.append({ "params":p[1] ,"lr":lr})
        if name.split(".")[1] == "pooler":
            for p in model.named_parameters():
                if p[0] == name and p[1].requires_grad:
                    parameters.append({ "params":p[1] ,"lr":lr_disc})
                    list1.append(p[0])
        if name.split(".")[1] == "embeddings":
            for p in model.named_parameters():
                if p[0] == name and p[1].requires_grad:
                    parameters.append({ "params":p[1] ,"lr":lr_disc})
        if name.split(".")[1] == "encoder":
                current = name.split('.')[3]
                if current != previous:
                    lr_disc = theta*lr_disc
                previous = current
                for p in model.named_parameters():
                    if p[0] == name and p[1].requires_grad:
                        parameters.append({ "params":p[1] ,"lr":lr_disc})
                        list1.append(p[0])
    return parameters, list1
#-----DISCRIMINATIVE-FINE-TUNING-----#
parameter,list1 = disc_fine_tuning(1)
#print(parameter)
optimizer = torch.optim.Adam(parameter)


In [126]:
for param_group in optimizer.param_groups:
        param_group = parameter

In [127]:
#-----MISC-----#
checkpoint = 0
torch.set_float32_matmul_precision("high")
#-----MISC-----#

In [131]:
for step in range(max_steps):
    last_step = (step == max_steps - 1)
    t0 = time.time()
    #optimizer.zero_grad(set_to_none=True)
    
    #validation loss
    if step % 250 == 1 or last_step:
        model.eval()
        with torch.no_grad():
            correct_acum = 0
            val_accum = 0
            val_loss_steps = 80
            for val_steps in range(val_loss_steps):
                x, y = next(iter(test_loader))
                x,y = x.to(device), y.to(device)
                #with torch.autocast(device_type="cuda", dtype=torch.dfloat16):
                logits,loss = model(x,y)
                values,ind = torch.max(logits,dim = 1)
                correct = np.sum((torch.eq(ind.to("cpu"),y.to("cpu")).numpy()))
                correct_acum = correct_acum + correct
                val_accum = val_accum + loss.detach()
            accuracy = correct_acum / (val_loss_steps * B)
            val_loss = val_accum / val_loss_steps
    
    #save checkpoint
    if step % 500 == 0:
        torch.save(model.state_dict(),f"checkpoint{checkpoint}.pt")
        checkpoint = checkpoint+1

    #minibatch process
    loss_accum = 0
    model.train()
    for mini_step in range(grad_accum_steps):
        x, y = next(iter(train_loader))
        x,y = x.to(device), y.to(device)
        #with torch.autocast(device_type="cuda", dtype=torch.dfloat16):
        logits,loss = model(x,y)
        #print(f"loss mini_step = {loss} | ministep = {mini_step}")
        loss = loss / grad_accum_steps
        loss_accum = loss_accum + loss.detach()
        loss.backward()
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 7.0)

    t1 = time.time()
    dt = t1-t0

    lr = get_lr(step)
    for param_group in optimizer.param_groups:
        param_group = parameter
    optimizer.step()

torch.save(model.state_dict(),"Final.pt")

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


KeyboardInterrupt: 

# misc

In [ ]:
#-----OPTIMIZER-----#
def configure_optimizers(model, weight_decay, learning_rate, device):
    param_dict = {pn: p for pn, p in model.named_parameters()}
    param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
    decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
    nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
    optim_groups = [
        {'params': decay_params, 'weight_decay': weight_decay},
        {'params': nodecay_params, 'weight_decay': 0.0}
    ]
    num_decay_params = sum(p.numel() for p in decay_params)
    num_nodecay_params = sum(p.numel() for p in nodecay_params) 
    print(num_decay_params,num_nodecay_params)
    fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
    use_fused = fused_available and device == "cuda"
    print(f"using fused AdamW: {use_fused}")
    optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=(0.9, 0.999), eps=1e-8, fused=use_fused)
    return optimizer  
  
optimizer = configure_optimizers(model = model, weight_decay=0.05, learning_rate=6e-4, device=device)
#-----OPTIMIZER-----#

In [ ]:
for param in model.named_parameters():
    print(param[0])

model_bert.embeddings.word_embeddings.weight
model_bert.embeddings.position_embeddings.weight
model_bert.embeddings.token_type_embeddings.weight
model_bert.embeddings.LayerNorm.weight
model_bert.embeddings.LayerNorm.bias
model_bert.encoder.layer.0.attention.self.query.weight
model_bert.encoder.layer.0.attention.self.query.bias
model_bert.encoder.layer.0.attention.self.key.weight
model_bert.encoder.layer.0.attention.self.key.bias
model_bert.encoder.layer.0.attention.self.value.weight
model_bert.encoder.layer.0.attention.self.value.bias
model_bert.encoder.layer.0.attention.output.dense.weight
model_bert.encoder.layer.0.attention.output.dense.bias
model_bert.encoder.layer.0.attention.output.LayerNorm.weight
model_bert.encoder.layer.0.attention.output.LayerNorm.bias
model_bert.encoder.layer.0.intermediate.dense.weight
model_bert.encoder.layer.0.intermediate.dense.bias
model_bert.encoder.layer.0.output.dense.weight
model_bert.encoder.layer.0.output.dense.bias
model_bert.encoder.layer.0.outp